In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sys, os
sys.path.append(os.path.dirname(sys.path[0]))

import numpy as np
import pandas as pd

import sys, os
sys.path.append(os.path.dirname(sys.path[0]))

from discrepancies import datasets, pool, pool2graph, evaluation

In [3]:
_DATASET = 'boston'
_K_INIT = 25
_K_REFINEMENT = 0
_MAX_EPOCHS = 5

X_train, X_test, y_train, y_test, scaler, feature_names, target_names = datasets.get_dataset(dataset=_DATASET, n_samples=1000, noise=0.3)

pool1 = pool.BasicPool()
#pool1 = pool.AutoSklearnPool(max_delta_accuracies=0.05, time_left_for_this_task=30, n_jobs=14)
pool1 = pool1.fit(X_train, y_train)

p2g = pool2graph.pool2graph(X_train,
                            y_train,
                            pool1,
                            k_init=_K_INIT,
                            k_refinement=_K_REFINEMENT)

p2g.fit(max_epochs=_MAX_EPOCHS)

In [22]:
tmp = {}
for i,n in lnodes:
    tmp[i] = n['features']
    
nodes_features = pd.DataFrame(tmp).T

In [8]:
from sklearn.neighbors import NearestNeighbors

k = 10

# (1) Process points with discrepancies: their nearest neighbors can be any points, no matter the discrepancies or the predicted labels

lnodes_index = lnodes_DP[lnodes_DP.discrepancies==1].index

_NN = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
_NN = _NN.fit(nodes_features)

_distances, _indices = _NN.kneighbors(lnodes_features.loc[lnodes_index])

# (2) Process points without discrepancies: their nearest neighbors can be points any points, EXCEPT points with the same predicted label

for p in lnodes_DP.pred.unique():

    _NN = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
    _NN = _NN.fit(nodes_features[~(lnodes_DP.pred==0)])
    
    lnodes_index = lnodes_DP[(lnodes_DP.discrepancies==0) & (lnodes_DP.pred==p)].index
    _distances, _indices = _NN.kneighbors(nodes_features.loc[lnodes_index])


In [27]:
np.concatenate((_distances,_distances),axis=0).shape

(18764, 11)

In [28]:
np.min([0,1])

0

In [13]:
lnodes_DP.discrepancies==0

 0         True
 1         True
 2        False
 3         True
 4         True
          ...  
-13890    False
-13891    False
-13892    False
-13893     True
-13894    False
Name: discrepancies, Length: 14233, dtype: bool